<a href="https://colab.research.google.com/github/harjotsinghparmar/Google-Colab-Deep-Learning/blob/master/CNN_concatenated_LBP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network

In this second exercise-notebook we will play with Convolutional Neural Network (CNN). 

As you should have seen, a CNN is a feed-forward neural network tipically composed of Convolutional, MaxPooling and Dense layers. 

If the task implemented by the CNN is a classification task, the last Dense layer should use the **Softmax** activation, and the loss should be the **categorical crossentropy**.

Reference: [https://github.com/fchollet/keras/blob/master/examples/cifar10_cnn.py]()

# Training the network

We will train our network on the **CIFAR10** [dataset](https://www.cs.toronto.edu/~kriz/cifar.html), which contains `50,000` 32x32 color training images, labeled over 10 categories, and 10,000 test images. 

As this dataset is also included in Keras datasets, we just ask the `keras.datasets` module for the dataset.

Training and test images are normalized to lie in the $\left[0,1\right]$ interval.

In [0]:
from keras.datasets import cifar10
from keras.utils import np_utils
%matplotlib inline
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation, Input, Concatenate
from keras.layers.merge import concatenate

nb_classes = 10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255
X_test /= 255

Using TensorFlow backend.


170500096/170498071 [==============================] - 21s 0us/step


In [0]:
## import libraries
import os 
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern

def six_channel_lbp(image_path):
    """
    Takes input as image path and returns six channel numpy array
    
    """
    img = cv2.imread(image_path,0)
    image = (cv2.imread(image_path))
    print(image.shape)
    for (r,p) in [(i,i*8) for i in range(1,4)]: 
        lbp = local_binary_pattern(img,p,r, method="uniform")
        expanded_lbp = (np.expand_dims(lbp,axis=2))
        image = np.dstack((image,expanded_lbp))
    return(image)


def six_channel_lbp_from_array(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    for (r,p) in [(i,i*8) for i in range(1,3)]: 
        lbp = local_binary_pattern(gray,p,r, method="uniform")
        expanded_lbp = (np.expand_dims(lbp,axis=2))
        image = np.dstack((image,expanded_lbp))
    return(image)
  
def six_channel_lbp_hist(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    empty_lbp = []
    total_bins = 0
    for (r,p) in [(i,i*8) for i in range(1,6)]: 
        lbp = local_binary_pattern(gray ,p,r, method="uniform")
        histogram = np.histogram(lbp,bins = np.arange(p+1))[0]
        empty_lbp.extend(histogram)
        total_bins = total_bins+p
    lbp_hist = np.array(empty_lbp,dtype=np.uint8)
    return(lbp_hist,total_bins)



Add lbp channels

In [0]:
X_train_lbp = np.asarray([six_channel_lbp_hist(x)[0] for x in X_train])

X_test_lbp = np.asarray([six_channel_lbp_hist(x)[0] for x in X_test])

(50000, 120)

#### Warning: The following cells may be computational Intensive....

In [0]:
# Create the cnn + lbp combo


In [0]:
from keras.layers.merge import concatenate
def cnn_lbp_model():
  #CNN part 
  input1 = Input(shape = (32,32,3,))
    
  x = (Conv2D(128, (3, 3), activation='relu', padding='same'))(input1)   
  x = (Conv2D(128, (3, 3), activation='relu'))(x)    
  x = (MaxPooling2D(pool_size=(2, 2)))(x)
  x = (Dropout(0.25))(x)

    
  x = (Conv2D(128, (3, 3), activation='relu', padding='same'))(x)
  x = (Conv2D(128, (3, 3), activation='relu'))(x)
  x = (MaxPooling2D(pool_size=(2, 2)))(x)
  x = (Dropout(0.25))(x)
  x = (Flatten())(x)
  
  #Concatenate LBP layer
  input2 = Input((120,))
  conc = concatenate([x,input2])
  conc1 = (Dense(512, activation='relu'))(conc)
  conc1 = (Dropout(0.5))(conc1)
  
    
  output = (Dense(nb_classes, activation='softmax'))(conc1)
  model = Model(inputs = [input1,input2], outputs = output)
  
  
  
  
  model.summary()
  
  return model 
  
  
def cnn_model():
  #CNN part 
  input1 = Input(shape = (32,32,3,))
    
  x = (Conv2D(128, (3, 3), activation='relu', padding='same'))(input1)   
  x = (Conv2D(128, (3, 3), activation='relu'))(x)    
  x = (MaxPooling2D(pool_size=(2, 2)))(x)
  x = (Dropout(0.25))(x)

    
  x = (Conv2D(128, (3, 3), activation='relu', padding='same'))(x)
  x = (Conv2D(128, (3, 3), activation='relu'))(x)
  x = (MaxPooling2D(pool_size=(2, 2)))(x)
  x = (Dropout(0.25))(x)
  x = (Flatten())(x)
  
  #Concatenate LBP layer

  output = (Dense(nb_classes, activation='softmax'))(x)
  model = Model(inputs = [input1], outputs = output)
  
  
  
  
  model.summary()
  
  return model 
  

In [0]:

model = cnn_lbp_model()
cnn_model = cnn_model()

model.compile(optimizer = 'rmsprop',loss = 'binary_crossentropy',learning_rate = 0.001)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_50 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_85 (Conv2D)              (None, 32, 32, 128)  3584        input_50[0][0]                   
__________________________________________________________________________________________________
conv2d_86 (Conv2D)              (None, 30, 30, 128)  147584      conv2d_85[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_43 (MaxPooling2D) (None, 15, 15, 128)  0           conv2d_86[0][0]                  
__________________________________________________________________________________________________
dropout_66

In [0]:
checkpoint = ModelCheckpoint('best_model_simple.h5',  # model filename
                             monitor='val_loss', # quantity to monitor
                             verbose=0, # verbosity - 0 or 1
                             save_best_only= True, # The latest best model will not be overwritten
                             mode='auto') # The decision to overwrite model is made 
                                          # automatically depending on the quantity to monitor

In [0]:
model.compile(loss='categorical_crossentropy', # Better loss function for neural networks
              optimizer=Adam(lr=1.0e-4), # Adam optimizer with 1.0e-4 learning rate
              metrics = ['accuracy']) # Metrics to be evaluated by the model

cnn_model.compile(loss='categorical_crossentropy', # Better loss function for neural networks
              optimizer=Adam(lr=1.0e-4), # Adam optimizer with 1.0e-4 learning rate
              metrics = ['accuracy']) # Metrics to be evaluated by the model

In [0]:
model_details = model.fit([X_train,X_train_lbp], Y_train,
                    batch_size = 128, # number of samples per gradient update
                    epochs =5000, # number of iterations
                    validation_data= ([X_test,X_test_lbp], Y_test),
                    callbacks=[checkpoint],
                    verbose=1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/5000
50000/50000 [==============================] - 35s 706us/step - loss: 3.1972 - acc: 0.2027 - val_loss: 1.8595 - val_acc: 0.3248
Epoch 2/5000
50000/50000 [==============================] - 33s 654us/step - loss: 1.8787 - acc: 0.3069 - val_loss: 1.6661 - val_acc: 0.4145
Epoch 3/5000
50000/50000 [==============================] - 33s 652us/step - loss: 1.7170 - acc: 0.3715 - val_loss: 1.5205 - val_acc: 0.4596
Epoch 4/5000
50000/50000 [==============================] - 33s 654us/step - loss: 1.6123 - acc: 0.4162 - val_loss: 1.4085 - val_acc: 0.4913
Epoch 5/5000
50000/50000 [==============================] - 33s 655us/step - loss: 1.5253 - acc: 0.4502 - val_loss: 1.3425 - val_acc: 0.5190
Epoch 6/5000
50000/50000 [==============================] - 33s 655us/step - loss: 1.4694 - acc: 0.4700 - val_loss: 1.3119 - val_acc: 0.5250
Epoch 7/5000
50000/50000 [==============================] - 33s 655us/step - loss: 1.4189 - acc: 0.4904 